In [7]:
# Problem 1 - Build the BWT

def makeBwt(t):
  """Create the BWT for the string t$"""
  
  # Code to complete
  td = t + "$"
  bwa = [i - 1 for i in range(len(td))]
  bwa.sort(key = lambda x : td[x + 1:])
  bwt = str()
  for k in bwa: bwt += td[k]

  return bwt

text = "GATTACA"

bwt = makeBwt(text)

bwt == "ACTGA$TA"

True

In [6]:
# Problem 2 - Invert the BWT

def invertBwt(bwt):
  """Inverts the Burrows-Wheeler Transform, returning the original string using 
  inefficent algorithm"""
  
  # Code to complete
  ## Hint - see the lecture notes
  inv_bwt = [bwt[i] for i in range(len(bwt))]

  for i in range(len(bwt) - 1):
    inv_bwt.sort()
    inv_bwt = [bwt[i] + inv_bwt[i] for i in range(len(bwt))]

  inv_bwt.sort(key = lambda x:x[-1])
  return inv_bwt[0] 

invertBwt(bwt) == text + "$"

True

In [5]:
# Problem 3 - Complete Last-to-First mapping using FM-index

class FmIndex(object):
    def __init__(self, t, alphabet):
      ''' Create FM-index for t in naive manner '''
      
      # Make the BWT 
      # We don't compress or anything here, but you could
      self.bwt = makeBwt(t)
      
      # Calculate C lookup dictionary in crappy way
      s = sorted(self.bwt)
      self.C = {}
      for i in range(len(s)-1, -1, -1):
        self.C[s[i]] = i
      
      # Calculate full Occ table in similarly crappy way
      # Recall, this is not using sampling and is therefore
      # very memory inefficient 
      self.Occ = [ {} for i in range(len(self.bwt)) ]
      for i in range(len(self.bwt)):
        for j in alphabet + "$":
          p = self.Occ[i-1][j] if i > 0 else 0
          self.Occ[i][j] = p + (1 if self.bwt[i] == j else 0)

      
    def lf(self, i):
      """ Return the last-to-first mapping for index i of the bwt """
      
      ## Code to complete
      # Hint, don't forget to convert between 0 based Python string coordinates and 1
      # based coordinates of FM-index tables
      # get out the F indicy
      last_item = self.bwt[i]
      first_item = self.C[last_item] + self.Occ[i][last_item] - 1
      return first_item
      

dnaAlphabet = "ACGT"
fmIndex = FmIndex(text, dnaAlphabet)

fmIndex.lf(5) == 0

True

In [8]:
# Problem 4 - Use backward search to invert the BWT

def invertBwtUsingFmIndex(fmIndex):
  """ Returns t by using repeated lf search to reconstruct t$ backwards"""
  
  # Code to write
  ## Hint: start from location of "$"  in bwt, then walk backwards using fmIndex.lf
  ## function to build t
  some_index = fmIndex.bwt.index('$')
  path = ''
  for postion in range(len(fmIndex.bwt)):
    path += bwt[some_index]
    some_index = fmIndex.lf(some_index)

  return path[::-1]
  

invertBwtUsingFmIndex(fmIndex) == "GATTACA$"

True

Problem 4 questions:

Q: What is the asymptotic runtime of your invertBwtUsingFmIndex function?

O($n^2$)

Q2: How does this to compare to the runtime of the invertBWT function?

invertBWT is slower than using FmIndex.

In [0]:
# Extra credit: adapt the FmIndex class and add a pattern query method to search for instance of input pattern in t